# pandas

> Stata-to-pandas utilities, used in `nbstata.browse`

In [ ]:
#| default_exp pandas
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.config import launch_stata
from nbstata.helpers import *
from nbstata.utils import *
import pandas as pd
import numpy as np

## Stata-to-pandas

In [ ]:
#| export
def simple_dataframe_from_stata(stfr, var, valuelabel, missingval):
    from pystata import stata
    if stfr is None:
        df = stata.pdataframe_from_data(var=var, valuelabel=valuelabel, missingval=missingval)
    else:
        df = stata.pdataframe_from_frame(stfr, var=var, valuelabel=valuelabel, missingval=missingval)
    df.index += 1
    return df

Reference Stata docs: [getAsDict example](https://blog.stata.com/2020/11/05/stata-python-integration-part-8-using-the-stata-function-interface-to-copy-data-from-stata-to-python/)

In [ ]:
#| export
def _better_dataframe(hdl, var, obs, selectvar, valuelabel, missingval):
    with IndexVar() as idx_var:
        data = hdl.getAsDict(var, obs, selectvar, valuelabel, missingval)
        if not data:
            return pd.DataFrame()
    
        if idx_var in data:
            idx = data.pop(idx_var)
        else:
            temp_var = [idx_var, selectvar] if selectvar else idx_var
            idx = hdl.getAsDict(temp_var, obs, selectvar, valuelabel, missingval).pop(idx_var)
        idx = pd.array(idx, dtype='int64')

        return pd.DataFrame(data=data, index=idx)

In [ ]:
#| hide
#| eval: False
launch_stata(splash=False)

In [ ]:
#| hide
#| eval: False
import sfi

In [ ]:
#| hide
from fastcore.test import test_eq, test_fail

In [ ]:
#| hide
#| eval: False
run_noecho("""\
sysuse uslifeexp2, clear
gen sel_var = year==1920""")

(U.S. life expectancy, 1900-1940)


In [ ]:
#| hide
#| eval: False
_better_dataframe(sfi.Data, 'le', None, 'sel_var', True, np.NaN)

,le
21,54.099998


In [ ]:
#| hide
#| eval: False
_better_dataframe(sfi.Data, None, range(0, 10), 'sel_var', True, np.NaN)

""


In [ ]:
#| hide
#| eval: False
_better_dataframe(sfi.Data, None, range(-31, -1), 'sel_var', True, np.NaN)

,year,le,sel_var
21,1920,54.099998,1.0


In [ ]:
#| hide
#| eval: False
run_noecho("drop sel_var")

In [ ]:
#| export
def _var_from_varlist(varlist, stfr):
    from pystata import stata
    if stfr:
        var_code = varlist.strip()
    else:
        _program_name = "temp_nbstata_varlist_name"
        try:
            var_code = diverted_stata_output(f"""\
                program {_program_name}
                    syntax [varlist(default=none)]
                    disp "`varlist'"
                end
                {_program_name} {varlist}
                program drop {_program_name}
                """).strip()
        except SystemError as e:
            stata.run(f"capture program drop {_program_name}", quietly=True)
            raise(e)
    return [c.strip() for c in var_code.split() if c] if var_code else None

In [ ]:
#| export
def better_dataframe_from_stata(stfr, varlist, obs, selectvar, valuelabel, missingval, sformat):
    import sfi, pystata
    hdl = sfi.Data if stfr is None else sfi.Frame.connect(stfr)
    var = _var_from_varlist(varlist, stfr)
    custom_index_not_needed = obs is None and not selectvar
    if custom_index_not_needed:
        df = simple_dataframe_from_stata(stfr, var, valuelabel, missingval)
    else:
        if hdl.getObsTotal() <= 0:
            return pd.DataFrame()
        df = _better_dataframe(hdl, var, obs, selectvar, valuelabel, missingval)
    if sformat:
        for v in list(df.columns):
            if hdl.isVarTypeString(v) or (valuelabel and missingval==np.NaN
                                          and pd.api.types.is_string_dtype(df[v])):
                continue
            v_format = hdl.getVarFormat(v)
            if missingval != np.NaN and pd.api.types.is_string_dtype(df[v]):
                def format_value(x):
                    return sfi.SFIToolkit.formatValue(x, v_format) if type(x)!=str else x
            else:
                def format_value(x):
                    return sfi.SFIToolkit.formatValue(x, v_format)
            df[v] = df[v].apply(format_value)
    return df

In [ ]:
#| export
def better_pdataframe_from_data(varlist="", obs=None, selectvar=None, valuelabel=False, missingval=np.NaN, sformat=False):
    import pystata
    pystata.config.check_initialized()

    return better_dataframe_from_stata(None, varlist, obs, selectvar, valuelabel, missingval, sformat)

In [ ]:
#| export
def better_pdataframe_from_frame(stfr, varlist="", obs=None, selectvar=None, valuelabel=False, missingval=np.NaN, sformat=False):
    import pystata
    pystata.config.check_initialized()

    return better_dataframe_from_stata(stfr, varlist, obs, selectvar, valuelabel, missingval, sformat)

In [ ]:
#| eval: False
run_noecho("""\
replace le = . if _n==5
replace year = 2022 if year==1900
format year %-ty
gen str_var = "test string" if _n!=5
gen date = dofy(year)
format date %td
gen double date_tc = cofd(date)
format date_tc %tc
gen double date_tC = Cofd(date)
format date_tC %tC
""")
better_pdataframe_from_data().head()

(1 real change made, 1 to missing)
(1 real change made)
(1 missing value generated)


,year,le,str_var,date,date_tc,date_tC
1,2022,47.299999,test string,22646.0,1.956614e+12,1.956614e+12
2,1901,49.099998,test string,-21549.0,-1.861834e+12,-1.861834e+12
3,1902,51.500000,test string,-21184.0,-1.830298e+12,-1.830298e+12
4,1903,50.500000,test string,-20819.0,-1.798762e+12,-1.798762e+12
5,1904,NaN,,-20454.0,-1.767226e+12,-1.767226e+12


In [ ]:
#| eval: False
better_pdataframe_from_data(sformat=True).head()

,year,le,str_var,date,date_tc,date_tC
1,2022,47.3,test string,01jan2022,01jan2022 00:00:00,01jan2022 00:00:00
2,1901,49.1,test string,01jan1901,01jan1901 00:00:00,01jan1901 00:00:00
3,1902,51.5,test string,01jan1902,01jan1902 00:00:00,01jan1902 00:00:00
4,1903,50.5,test string,01jan1903,01jan1903 00:00:00,01jan1903 00:00:00
5,1904,.,,01jan1904,01jan1904 00:00:00,01jan1904 00:00:00


In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(valuelabel=True, sformat=True, missingval=".").head()

,year,le,str_var,date,date_tc,date_tC
1,2022,47.3,test string,01jan2022,01jan2022 00:00:00,01jan2022 00:00:00
2,1901,49.1,test string,01jan1901,01jan1901 00:00:00,01jan1901 00:00:00
3,1902,51.5,test string,01jan1902,01jan1902 00:00:00,01jan1902 00:00:00
4,1903,50.5,test string,01jan1903,01jan1903 00:00:00,01jan1903 00:00:00
5,1904,.,,01jan1904,01jan1904 00:00:00,01jan1904 00:00:00


In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(sformat=True, missingval=".").head()

,year,le,str_var,date,date_tc,date_tC
1,2022,47.3,test string,01jan2022,01jan2022 00:00:00,01jan2022 00:00:00
2,1901,49.1,test string,01jan1901,01jan1901 00:00:00,01jan1901 00:00:00
3,1902,51.5,test string,01jan1902,01jan1902 00:00:00,01jan1902 00:00:00
4,1903,50.5,test string,01jan1903,01jan1903 00:00:00,01jan1903 00:00:00
5,1904,.,,01jan1904,01jan1904 00:00:00,01jan1904 00:00:00


In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(obs=range(1, 3)).head()

,year,le,str_var,date,date_tc,date_tC
2,1901,49.099998,test string,-21549.0,-1.861834e+12,-1.861834e+12
3,1902,51.500000,test string,-21184.0,-1.830298e+12,-1.830298e+12


In [ ]:
#| hide
#| eval: False
test_fail(better_pdataframe_from_data, kwargs=dict(obs=range(1000)))

In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(missingval='.').head()

,year,le,str_var,date,date_tc,date_tC
1,2022,47.299999,test string,22646.0,1.956614e+12,1.956614e+12
2,1901,49.099998,test string,-21549.0,-1.861834e+12,-1.861834e+12
3,1902,51.5,test string,-21184.0,-1.830298e+12,-1.830298e+12
4,1903,50.5,test string,-20819.0,-1.798762e+12,-1.798762e+12
5,1904,.,,-20454.0,-1.767226e+12,-1.767226e+12


In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(varlist='y date_*').head()

,year,date_tc,date_tC
1,2022,1.956614e+12,1.956614e+12
2,1901,-1.861834e+12,-1.861834e+12
3,1902,-1.830298e+12,-1.830298e+12
4,1903,-1.798762e+12,-1.798762e+12
5,1904,-1.767226e+12,-1.767226e+12


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()